In [19]:
import pandas as pd
import folium
import branca

# Load the data
df = pd.read_excel("146425_00_5 (1).xlsx", sheet_name=1)

# Convert the Latitude and Longitude columns to numeric values, coercing errors to NaN
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

# Drop rows with missing Latitude or Longitude
df = df.dropna(subset=['Latitude', 'Longitude'])

# Create a map centered around Australia
m = folium.Map(location=[-25.2744, 133.7751], zoom_start=4)

# Define the color based on the status
def color_producer(status):
    if status == 'Operating':
        return 'green'
    elif status == 'In development':
        return 'orange'
    else:
        return 'red'

# Add a circle marker for each project
for idx, row in df.iterrows():
    popup_text = (
        f"<div style='width: 150px; white-space: normal;'>"
        f"<b>Proponents:</b> {row['Proponents']}<br>"
        f"<b>Electrolyser Size:</b> {row['Electrolyser Size (MW)']} MW<br>"
        f"<b>Expected Start Date:</b> {row['Expected Start Date']}<br>"
        f"<b>Reference:</b> <a href='{row['Reference']}' target='_blank'>Link</a>"
        f"</div>"
    )
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=4,  # you can control the size by setting the radius
        color=color_producer(row['Status']),
        fill=True,
        fill_opacity=1,
        popup=folium.Popup(popup_text, max_width=250)
    ).add_to(m)

# Create a legend
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; width: 120px; height: 125px; 
border:2px solid grey; z-index:9999; font-size:14px; background:white; 
opacity: 0.7;">
<p><i class="fa fa-circle fa-1x" style="color:green"></i>&nbsp; Operating</p>
<p><i class="fa fa-circle fa-1x" style="color:orange"></i>&nbsp; In Development</p>
<p><i class="fa fa-circle fa-1x" style="color:red"></i>&nbsp; Under Construction</p>
</div>
"""
m.get_root().html.add_child(branca.element.Element(legend_html))

# Save the map to an HTML file
m.save('map.html')
